In [1]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
    %config Completer.use_jedi = False
except:
    pass

# Basic packages
import numpy as np
import pandas as pd
np.seterr(divide='ignore', invalid='ignore')
from astropy.table import Table, join
from astropy.io import fits
from scipy.io import readsav
from astropy.io import ascii

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt





In [2]:
# read in the agh table
agh = Table.read('agh_final.fits')

In [4]:
teff = agh['FPARAM'][:,0]
logg = agh['FPARAM'][:,1]
vmic = 10**agh['FPARAM'][:,2]
fe_h = agh['FPARAM'][:,3]
a_fe = agh['FPARAM'][:,6]
vsini = 10**agh['FPARAM'][:,7]

In [20]:
uncalibrated_apogee = agh['tmass_id','teff']

In [21]:
uncalibrated_apogee['TEFF']=teff
uncalibrated_apogee['LOGG']=logg
uncalibrated_apogee['FE_H']=fe_h
uncalibrated_apogee['VMICRO']=vmic
uncalibrated_apogee['VSINI']=vsini
uncalibrated_apogee.remove_column('teff')

FELEM: (C, CI, N, O, Na, Mg, Al, Si, P, S, K, Ca, Ti, TiII, V, Cr, Mn, Fe, Co, Ni, Cu, Ge, Rb, Ce, Nd, Yb, C13)


In [44]:
# remvoe CI, TiII, C13
# index for other elements
uncal_ele = np.array(agh['FELEM'])
ele_indices = [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

# Select the desired columns
ele_arr = uncal_ele[:, ele_indices]

# These are the names of the columns you want to keep
column_names = ['C', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K', \
                'Ca', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Ge', 'Rb', 'Ce', 'Nd', 'Yb']

# Convert the array to a pandas DataFrame
ele_uncalibrated = pd.DataFrame(ele_arr, columns=column_names)

In [48]:
# fill the nan value of the table by 0 
uncal_abund = ele_uncalibrated.fillna(0)

In [53]:
def calculate_sme_abundance_table(sample):
    '''
    input: the elemental abundance measured by GALAH and APOGEE (pandas dataframe)
    output: sme.abund
    '''
    
    #       "H",  "He",  "Li",  "Be",   "B",   "C",   "N",   "O",   "F",  "Ne",
    a_x= [12.00, 10.93,  1.05,  1.38,  2.70,  8.39,  7.78,  8.66,  4.56,  7.84,
    #      "Na",  "Mg",  "Al",  "Si",   "P",   "S",  "Cl",  "Ar",   "K",  "Ca",
           6.17,  7.53,  6.37,  7.51,  5.36,  7.14,  5.50,  6.18,  5.08,  6.31,
    #      "Sc",  "Ti",   "V",  "Cr",  "Mn",  "Fe",  "Co",  "Ni",  "Cu",  "Zn",
           3.17,  4.90,  4.00,  5.64,  5.39,  7.45,  4.92,  6.23,  4.21,  4.60,
    #      "Ga",  "Ge",  "As",  "Se",  "Br",  "Kr",  "Rb",  "Sr",   "Y",  "Zr",
           2.88,  3.58,  2.29,  3.33,  2.56,  3.25,  2.60,  2.92,  2.21,  2.58,
    #      "Nb",  "Mo",  "Tc",  "Ru",  "Rh",  "Pd",  "Ag",  "Cd",  "In",  "Sn",
           1.42,  1.92, -8.00,  1.84,  1.12,  1.66,  0.94,  1.77,  1.60,  2.00,
    #      "Sb",  "Te",   "I",  "Xe",  "Cs",  "Ba",  "La",  "Ce",  "Pr",  "Nd",
           1.00,  2.19,  1.51,  2.24,  1.07,  2.17,  1.13,  1.70,  0.58,  1.45,
    #      "Pm",  "Sm",  "Eu",  "Gd",  "Tb",  "Dy",  "Ho",  "Er",  "Tm",  "Yb",
          -8.00,  1.00,  0.52,  1.11,  0.28,  1.14,  0.51,  0.93,  0.00,  1.08,
    #      "Lu",  "Hf",  "Ta",   "W",  "Re",  "Os",  "Ir",  "Pt",  "Au",  "Hg",
           0.06,  0.88, -0.17,  1.11,  0.23,  1.25,  1.38,  1.64,  1.01,  1.13,
    #      "Tl",  "Pb",  "Bi",  "Po",  "At",  "Rn",  "Fr",  "Ra",  "Ac",  "Th",
           0.90,  2.00,  0.65, -8.00, -8.00, -8.00, -8.00, -8.00, -8.00,  0.06,
    #      "Pa",   "U",  "Np",  "Pu",  "Am",  "Cm",  "Bk",  "Cf",  "Es", "TiO" 
          -8.00, -0.52, -8.00, -8.00, -8.00, -8.00, -8.00, -8.00, -8.00]
   
    log_eonh = np.array(a_x) - 12.0 #Default abundance pattern in sme.abund?
    log_eonfe = np.array(a_x)-7.45 # convert to log(element/Fe)
    
    
    elements = np.array(['C', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K', 'Ca', 'Ti', \
                         'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Ge', 'Rb', 'Ce', 'Nd', 'Yb']) # elements measured in the survey

    element_indices = np.array([6, 7, 8, 11, 12, 13, 14, 15, 16, 19, 20, 22, \
                                23, 24, 25, 26, 27, 28, 29, 32, 37, 58, 60, 70])-1 

    
    log_eonh_ = []
    abund_list = []
    
        
        
    for i in range(len(sample)): # for each star 
        logeonh=list(log_eonh)
        for j in range(len(elements)): # index and element
            element = elements[j] # element
            element_index = element_indices[j] # index of the element in a_x 
            logeonh[element_index] = log_eonh[element_index] + sample[element][i] # get the log element on h 
        
        
        
        
        eonh = 10**np.array(logeonh) # the element on hydrogen
        renorm = np.sum(eonh)

        eontot = eonh / renorm
        log_eontot = np.log10(eontot).astype(np.float)
        abund = log_eontot
        abund[0] = eonh[0] / renorm
        abund = abund.astype(np.float)
        
        abund_list.append(abund)

    return(abund_list)
    
    
    
    '''
    for index, element in enumerate(elements):
        element_index = element_indices[index] # element index in a_x
        logeonh=list(log_eonh)
        logeonh[element_index] = log_eonh[element_index] + np.array(sample[element+'_FE']) # log element on Fe of the observed star
    '''
  
    

In [54]:
uncalibrated_abund = calculate_sme_abundance_table(uncal_abund)

/Users/anne/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/Users/anne/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [61]:
uncalibrated_apogee['SME_abund'] = uncalibrated_abund

In [62]:
uncalibrated_apogee

tmass_id,TEFF,LOGG,FE_H,VMICRO,VSINI,SME_abund [99]
bytes17,float32,float32,float32,float32,float32,float64
00041790-7249588,4950.8,3.4213,-0.15789,1.0906038,3.2427979,0.9208075029068955 .. -20.035831150653472
00050927-5144526,4550.4,2.5531,0.05236,1.2626413,2.8674467,0.9205774528816895 .. -20.035939666210435
00063954-2027036,4770.8,3.4626,0.32945,0.3010163,2.4382977,0.9207290598375084 .. -20.03586814953027
00074341-2000506,4564.0,2.5997,0.062292,1.3086092,2.850832,0.9207385353286056 .. -20.03586368010218
00081109-2105478,4778.5,3.4316,0.1529,0.96628004,2.7036355,0.9207482306752444 .. -20.03585910701958
00082727-5205478,4502.1,2.6266,-0.25849,1.3157398,3.439397,0.9204049118290788 .. -20.036021072343058
...,...,...,...,...,...,...
23534868+1643361,4872.9,2.6039,-0.44598,1.513631,3.8381643,0.9202271359183576 .. -20.036104964283737
23535446+1626538,4721.7,2.5697,-0.43951,1.3159215,3.8236618,0.9201670386482085 .. -20.03613332767803
